In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())


print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
2
0
NVIDIA GeForce RTX 3090


In [13]:
# iv: image option
length = 250
channel = 128
min_CNN = 200
n_classes = 4
classes = range(n_classes)


In [6]:
# if platform == "linux" or platform == "linux2":
#     torch_models_dir = r"/media/titan/AI Research1/Data/CVPR2017"
# elif platform == "win32":
#     torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
# block_splits_all, block_splits_single, eeg_14_70, eeg_55_95, eeg_5_95, eeg_raw = os.listdir(torch_models_dir)
# print(os.listdir(torch_models_dir))

In [5]:
eeg_dataset = '/media/mountHDD1/LanxHuyen/high_gamma_dataset.pth'

In [8]:
# eeg_dataset = os.path.join(torch_models_dir, eeg_5_95)
# splits_all_path = os.path.join(torch_models_dir, block_splits_all)
# splits_single_path = os.path.join(torch_models_dir, block_splits_single)
# # splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
# print(eeg_dataset,'\n', splits_all_path, '\n', splits_single_path)

In [9]:
# splits_all = torch.load(splits_all_path)
# splits_single = torch.load(splits_single_path)

In [10]:
# print(len(splits_all['splits']))
# print(len(splits_all['splits'][0]))

# print(len(splits_all['splits'][5]['train']))
# print(len(splits_all['splits'][5]['val']))
# print(len(splits_all['splits'][5]['test']))
# print(splits_all['splits'][0]['train'][:40])
# print(splits_all['splits'][1]['train'][:40])
# print(splits_all['splits'][2]['train'][:10])
# print(splits_all['splits'][3]['train'][:10])


In [11]:
# # print(splits_single)
# print(len(splits_single['splits'][0]['train']))
# print(len(splits_single['splits'][0]['val']))
# print(len(splits_single['splits'][0]['test']))

In [8]:
eeg_loaded = torch.load(eeg_dataset)

In [9]:
indices = list(range(len(eeg_loaded)))
random.shuffle(list(range(len(eeg_loaded))))

In [10]:
train_size = int(0.8 * len(eeg_loaded))
val_size = int(0.1 * len(eeg_loaded))
test_size = len(eeg_loaded) - (train_size + val_size)

In [11]:
train_indices = indices[:train_size]
val_indices = indices[train_size:train_size+val_size]
test_indices = indices[train_size+val_size:]

In [12]:
loaded_splits = {"train": train_indices, "val": val_indices, "test": test_indices}

In [13]:
print(len(eeg_loaded))
# print(eeg_loaded.keys())
eeg = [eeg_loaded[i]["eeg"] for i in range (len(eeg_loaded))]
print(len(eeg))
# for dataset_idx in range (len(eeg_loaded)): 
#     eeg, label = [eeg_loaded[dataset_idx][key] for key in ['eeg', 'label']]
# print((label))
# print(len(eeg))
# # print(len(dataset))

# # print(labels)
# # print(images[0])
# print(eeg['eeg'][0].shape)
# print(eeg[0])

26968
26968


In [6]:
opt = {
    # Dataset options
#     "iv": "image",
#     "offset": None,
    "results_file": "results.pkl",
    #"subject": 0,
    "time_low": 20,
    "time_high": 270,
#     "run": "none",
    "eeg_dataset": eeg_dataset,
    "model_type": "model10",
    #"splits_path": splits_all_path,
    #"split_num": 0,
    "split_name": "train",
#     "fold": 5,
    #Training options
    "batch_size": 64,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"from-scratch",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.time_low)

20


In [7]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
# from data_loader import EEGDataset, Splitter, SplitterWithData
from data_loader_HGD import EEGDataset, Splitter
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

In [8]:
def load_dataset(
#              offset,
             eeg_dataset,
             #splits_path,
             #split_num, # (0-5) - 6 fold cross validation
             split_name,
#              total, 
#              classes,
#              classifier,
             batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             kind="from-scratch"):        
    # Load dataset
    dataset = EEGDataset(opt, eeg_dataset)
    print("DONE: LOAD DATASET")
#     # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
#     if kind=="from-scratch":
#         relabel = False
#     if kind=="incremental":
#         relabel = False
#     if kind=="no-model-file":
#         relabel = True
    splitter = {split: Splitter(dataset,
                    #splits_path,
                    #split_num,
                    split_name = split) 
                for split in ["train", "val", "test"]
                                }
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [9]:
# Options
opt.classifier = "LSTM4"
opt.batch_size = 64
# opt.kind = "from-scratch"
# opt.run = "imagenet40-1000"
# opt.fold = 5

In [10]:
dataset, loaders, splitter = load_dataset(
#              offset,
             opt.eeg_dataset,
             #opt.splits_path,
             #opt.split_num, # (0-5) - 6 fold cross validation
             opt.split_name,
#              total, 
#              classes,
#              classifier,
             opt.batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [11]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 8000 idx, val: 2000 idx, test: 2000 idx}
# Each loader batch has {train: 2000 idx, val: 250 idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i, (input, target) in enumerate(loaders["train"]):
    if i<20:
        print(f"{i+1}: Target size: {target.size()}; input size: {input.size()}")
# for i in range(0, 40):
#     eeg, label_val = splitter["val"][i]
#     eeg, label_train = splitter["train"][i]
#     print(f"{i+1}: Label val: {label_val}; label train: {label_train}")


<class 'data_loader_HGD.EEGDataset'>
<class 'dict'>
3 [338, 43, 43]
1: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
2: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
3: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
4: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
5: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
6: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
7: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
8: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
9: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
10: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
11: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
12: Target size: torch.Size([64]); input size: torch.Size([64, 128, 250])
13: Target size: torch.Size([64]); input size: torch.

In [14]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1,128, 250))

DONE: CREATE TORCH CLASSIFIER
classifier_LSTM(
  (lstm): LSTM(128, 128, batch_first=True)
  (output1): Linear(in_features=128, out_features=128, bias=True)
  (relu): ReLU()
  (output2): Linear(in_features=128, out_features=4, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_LSTM                          [1, 4]                    --
├─LSTM: 1-1                              [1, 250, 128]             132,096
├─Linear: 1-2                            [1, 128]                  16,512
├─ReLU: 1-3                              [1, 128]                  --
├─Linear: 1-4                            [1, 4]                    516
Total params: 149,124
Trainable params: 149,124
Non-trainable params: 0
Total mult-adds (M): 33.04
Input size (MB): 0.13
Forward/backward pass size (MB): 0.26
Params size (MB): 0.60
Estimated Total Size (MB): 0.98

In [15]:
model_path = (   opt.classifier+
                  " -" + "HGD" )
print(model_path)

LSTM4 -HGD


In [16]:
print(opt)

Namespace(results_file='results.pkl', time_low=20, time_high=270, eeg_dataset='/media/mountHDD1/LanxHuyen/high_gamma_dataset.pth', model_type='model10', split_name='train', batch_size=64, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='LSTM4')


In [17]:
channel_idx=None
non_classes=None
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

Epoch 1
Train Batch 100 (every 100 batch): Loss=1.2298; accuracy=0.3948
Train Batch 200 (every 100 batch): Loss=1.2076; accuracy=0.4248
Train Batch 300 (every 100 batch): Loss=1.0434; accuracy=0.4429
Epoch 1 summary: train_loss: 1.2020 | train_acc: 0.4476 | val_loss: 1.3066 | val_acc: 0.3921
Epoch 2
Train Batch 100 (every 100 batch): Loss=1.0772; accuracy=0.4881
Train Batch 200 (every 100 batch): Loss=1.1073; accuracy=0.4987
Train Batch 300 (every 100 batch): Loss=1.3118; accuracy=0.5039
Epoch 2 summary: train_loss: 1.1141 | train_acc: 0.5084 | val_loss: 1.2255 | val_acc: 0.4560
Epoch 3
Train Batch 100 (every 100 batch): Loss=1.2000; accuracy=0.5528
Train Batch 200 (every 100 batch): Loss=0.9269; accuracy=0.5454
Train Batch 300 (every 100 batch): Loss=1.1750; accuracy=0.5469
Epoch 3 summary: train_loss: 1.0658 | train_acc: 0.5461 | val_loss: 1.2254 | val_acc: 0.4509
Epoch 4
Train Batch 100 (every 100 batch): Loss=1.0318; accuracy=0.5455
Train Batch 200 (every 100 batch): Loss=1.0063; a

ValueError: too many values to unpack (expected 3)

In [ ]:
# val =accuracy_val
# test = accuracy_test

# print("Validation accuracy: ", val)
# print("Test accuracy: ", test)

In [ ]:
print(accuracy_val)